<a href="https://colab.research.google.com/github/kyriakos77kolovos/ergasies_python/blob/master/ASR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

model_utilities.py

In [ ]:
from os import listdir
from os.path import join
from librosa import resample, stft, magphase, amplitude_to_db
from librosa.feature import melspectrogram
from pathlib import Path
from pickle import load, dump
from warnings import filterwarnings

import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils
from pandas import DataFrame, read_csv
from seaborn import heatmap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow._api.v2.math import confusion_matrix
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.layers import LSTM, Dense, Dropout
from tensorflow.python.keras.models import Sequential, load_model

#from Utilities.Constants import PATH_OF_SAVED_MODEL, PATH_OF_SAVED_LABELS, PATH_OF_SAVED_MODEL_HISTORY
#from Utilities.load_files import loadTrainedData

filterwarnings("ignore")


def loadModel(model_path: Path):
    model = load_model(model_path)
    return model


def saveModel(model: Sequential, model_path: Path):
    model.save(model_path)


def loadHistory(model_history_path: Path):
    model_history = read_csv(model_history_path)
    return model_history


def saveHistory(history: DataFrame, model_history_path: Path):
    history.to_csv(model_history_path)


def loadLabelEncoder(labels_path: Path):
    label_encoder = load(open(labels_path, 'rb'))
    return label_encoder


def saveLabelEncoder(label_encoder: LabelEncoder, labels_path: Path):
    dump(label_encoder, open(labels_path, 'wb'))


def createModel(x_train: np.array, classes_count):
    input_shape = (x_train.shape[1], x_train.shape[2])

    model = Sequential()

    # Adding Layers
    # 1st LSTM layer
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    # 2nd LSTM layer
    model.add(LSTM(64))

    # 1st Dense layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    # Outputs
    model.add(Dense(classes_count, activation='softmax'))
    return model


def initCallbacks():
    es = EarlyStopping(monitor='val_loss',
                       mode='min',
                       patience=6,
                       min_delta=0.0001)

    mc = ModelCheckpoint('best_model.hdf5',
                         monitor='val_acc',
                         save_best_only=True,
                         mode='max')
    return es, mc


def splitDataset(mel_trained_data: np.array, y: np.array):
    x_train, x_test, y_train, y_test = train_test_split(mel_trained_data,
                                                        np.array(y),
                                                        stratify=y,
                                                        test_size=0.2,
                                                        shuffle=True)

    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                      test_size=0.2)

    x_train = np.array(x_train)
    x_test = np.array(x_test)
    x_val = np.array(x_val)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_val = np.array(y_val)

    return x_train, x_test, y_train, y_test, x_val, y_val


def modelEvaluation(model: Sequential, x_train, y_train, x_test, y_test):
    train_score = model.evaluate(x_train, y_train, batch_size=32)
    test_score = model.evaluate(x_test, y_test, batch_size=32)

    print("Train score: {}".format(train_score))
    print("Test score: {}".format(test_score))


def displayModelPlot(history_dict: DataFrame):
    loss_values = history_dict['loss']
    acc_values = history_dict['accuracy']
    val_loss_values = history_dict['val_loss']
    val_acc_values = history_dict['val_accuracy']
    epochs = range(1, len(loss_values) + 1)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.plot(epochs, loss_values, 'co', label='Training Loss')
    ax1.plot(epochs, val_loss_values, 'm', label='Validation Loss')
    ax1.set_title('Training and validation loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax2.plot(epochs, acc_values, 'co', label='Training accuracy')
    ax2.plot(epochs, val_acc_values, 'm', label='Validation accuracy')
    ax2.set_title('Training and validation accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    plt.show()
    plt.close()


def displayConfusionMatrix(model: Sequential, x_test, y_test, classes):
    y_predict = model.predict(x_test)
    conf_mat = confusion_matrix(np.argmax(y_test, axis=1),
                                np.argmax(y_predict, axis=1))

    df_cm = DataFrame(np.array(conf_mat), index=[i for i in classes],
                         columns=[i for i in classes])
    plt.figure(figsize=(13, 7))
    ax = heatmap(df_cm, annot=True)
    plt.title("Confusion Matrix", fontsize=20)
    plt.ylabel("True Class", fontsize=20)
    plt.xlabel("Predicted Class", fontsize=20)
    plt.show()
    plt.close()


def createModels():
    y, classes = None, None
    model_path = Path("/drive/MyDrive/ASR/ASR_Project-main/")
    model_history_path = Path("/drive/MyDrive/ASR/ASR_Project-main/")
    labels_path = Path("/drive/MyDrive/ASR/ASR_Project-main/")

    TRAIN_DATA_PATH_FLAG_EXISTS, mel_trained_data, labels = loadTrainedData()
    model, model_history, label_encoder = None, None, None
    if model_path.exists():
        model = loadModel(model_path)

    if labels_path.exists():
        label_encoder = loadLabelEncoder(labels_path)
        y = label_encoder.fit_transform(labels)
        classes = list(label_encoder.classes_)
        classes_count = len(classes)
        y = np_utils.to_categorical(y, num_classes=classes_count, dtype='float32')

    if TRAIN_DATA_PATH_FLAG_EXISTS:
        x_train, x_test, y_train, y_test, x_val, y_val = splitDataset(mel_trained_data, y)
    else:
        return model, label_encoder

    if model_history_path.exists() and TRAIN_DATA_PATH_FLAG_EXISTS:
        model_history = loadHistory(model_history_path)
        displayModelPlot(model_history)
        displayConfusionMatrix(model, x_test, y_test, classes)

    if model is None or label_encoder is None:

        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(labels)
        classes = list(label_encoder.classes_)
        classes_count = len(classes)
        y = np_utils.to_categorical(y, num_classes=classes_count, dtype='float32')

        saveLabelEncoder(label_encoder, labels_path)

        if TRAIN_DATA_PATH_FLAG_EXISTS:
            x_train, x_test, y_train, y_test, x_val, y_val = splitDataset(mel_trained_data, y)
        else:
            return None, None

        model = createModel(np.array(x_train), classes_count)
        model.summary()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        es, mc = initCallbacks()

        model_history = model.fit(x_train, y_train,
                                  validation_data=(x_val, y_val),
                                  epochs=100, callbacks=[es, mc],
                                  batch_size=64)
        model_history = model_history.history
        saveModel(model, model_path)
        saveHistory(model_history, model_history_path)
        displayModelPlot(model_history)
        modelEvaluation(model, x_train, y_train, x_test, y_test)
        displayConfusionMatrix(model, x_test, y_test, classes)

    return model, label_encoder


def predictDigit(model: Sequential, classes, signal: np.array):
    prob = model.predict(signal)
    index = np.argmax(prob[0])
    return classes[index]


def loadTrainedData():
        return True, None, None

requirements.txt

In [ ]:
!pip3 install seaborn~=0.11.2